 # Explorando o Hacker News

O [Hacker News](https://news.ycombinator.com) é um site iniciado pela incubadora de startups Y Combinator, onde posts enviados por usuários  recebem votos e comentários. A plataforma é extremamente popular na área de tecnologia, e as postagens que chegam ao topo da listagem podem receber centenas de milhares de visitantes.

Os dados originais podem ser encontrados [aqui](https://www.kaggle.com/hacker-news/hacker-news-posts).
Para esta análise, eliminamos os envios que não receberam comentários e mostramos aleatoriamente os envios restantes. As 7 colunas são descritas abaixo:

* `id`


* `Title:`- Título


* `url`


* `num_points`:- Número de pontos que o post adquiriu 

 
* `num_comments`:- Número de comentários no post


* `author`:- Nome de usuário da pessoa que enviou o post


* `created_at`:- Data e hora do envio do post


## Abrindo e explorando 

In [1]:
from csv import reader
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Criaremos agora, uma função chamada `extrair_dados()` que recebe um caminho de arquivo como argumento e retorna uma lista de listas. A função retornará as colunas e os dados separadamente.

In [2]:
def extrair_dados(filepath):
    
    abrir_arq = open(filepath)
    ler_arq = reader(abrir_arq)
    resultado = list(ler_arq)
    
    return resultado[0], resultado[1:]

Vamos definir outra função chamada `explorar_dados()` para nos permitir exibir intervalos especificados de linhas em formato legível:

In [3]:
def explorar_dados(dataset, cmc, fim, l_e_c=False):
    
    parte_dados = dataset[cmc:fim]    
    for row in parte_dados:
        print(row)
        print('\n')

    if l_e_c:
        print('Número de linhas: {:,}'.format(len(dataset)))
        print('Número de colunas:', len(dataset[0]))

Agora vamos utilizar as funções que criamos e analisar esses dados:

In [4]:
# extrai o cabeçalho e os dados de 'hacker_news.csv'
hn_cbc, hn = extrair_dados('hacker_news.csv')

# imprime os cabeçalhos e explora as primeiras cinco linhas 
print(hn_cbc, '\n')

explorar_dados(hn, 0, 4, True)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


Número de linhas: 20,100
Número de colunas: 7


Os dados extraídos contêm um total de 20.100 linhas e 7 colunas. Porém algumas colunas serão extremamente úteis em nossa análise>:  - `title`, `num_points`, `num_comments` e `created_at`.


## Trabalhando com Ask HN e Show HN

Nesse momento, estamos interessados apenas em posts que começam com Ask HN ou Show HN. Assim, vamos criar novas listas contendo apenas os dados desses títulos e armazená-los nas variáveis correspondentes.

In [5]:
ask_posts = []
show_posts = []
outros_posts = []

for row in hn:
    title = row[1]

#Para encontrar as postagens que começam com Ask HN ou Show HN usaremos string.startswith()
#Usaremos o string.lower_method() para garantir que não haja diferença entre maiúsculas de minúsculas
    if title.lower().startswith('show hn'):
        show_posts.append(row)
    elif title.lower().startswith('ask hn'):
        ask_posts.append(row)
    else:
        outros_posts.append(row)

Vamos verificar se tudo deu certo:

In [6]:
template = '{}: {:,} ({:.0f}%)'

print(template.format('No de Ask HN posts', len(ask_posts), 100*len(ask_posts)/len(hn)))
print(template.format('No de Show HN posts', len(show_posts), 100*len(show_posts)/len(hn)))
print(template.format('No de Outros posts', len(outros_posts), 100*len(outros_posts)/len(hn)))

# verificando se nenhuma linha foi deixada sem categoria comparando os totais
print(len(ask_posts) + len(show_posts) + len(outros_posts) == len(hn))

No de Ask HN posts: 1,744 (9%)
No de Show HN posts: 1,162 (6%)
No de Outros posts: 17,194 (86%)
True


Como podemos ver, 9% são postagens de usuários que fazem perguntas específicas, 6% são postagens de  um determinado assunto e 86% dessas postagens não são nem perguntar nem mostrar postagens .

 Vamos nos aprofundar mais um pouco e começar calculando o número médio de comentários em cada categoria.

## Número médio de comentários para cada categoria

Para calcular o número médio de comentários, vamos definir uma função chamada `calcular_media()`:

In [7]:
def calcular_media(dataset, index):
    
#calcula e retorna a média de todos os valores da coluna index
    total = 0
    n = len(dataset)
    
    for row in dataset:
        total += int(row[index])
        
    resultado = total/n
    return round(resultado,2)

Chamaremos esta função para calcular as médias correspondentes para as postagens Show HN e Ask HN:

In [8]:
med_show = calcular_media(show_posts, 4)
med_ask = calcular_media(ask_posts, 4)

template = '{}: {} comentários em média'

print(template.format('Ask HN', med_ask))
print(template.format('Show HN', med_show))

Ask HN: 14.04 comentários em média
Show HN: 10.32 comentários em média


Podemos constatar então que  as postagens do Ask ganham mais interação do usuário, na forma de comentários, do que as postagens do show.

## Número de postagens e comentários por hora de criação

Agora, trabalharemos no cálculo do número total de postagens e comentários em cada hora nas duas categorias. Usaremos o datetime para trabalhar com os dados na coluna `created_at`. Então, usaremos o construtor `datetime.strptime()` para analisar datas armazenadas como strings em objetos datetime.
* Para isso vamos definir duas funções:
    - Uma função `ordenar_hora()` que recebe um dicionário com horas como chaves e classifica cada chave no dicionário.
    - Uma função `calcular_estat_tempo()`, que calcula estatísticas de tempo com base em um determinado índice. Esta função vai retornar dois dicionários: um contendo a frequência de postagens por hora, e outro contendo a soma do índice de medida (neste caso, o número de comentários) - agrupados por hora.

In [9]:
def ordenar_hora(dictionary):
    resultado = {}
    ord_alc = list(range(0, 24))
    for hora in ord_alc:
        resultado[hora] = dictionary[hora]
    
    return resultado

In [10]:
def calcular_estat_tempo(dataset, t_index, m_index):

    resultado_a = {}
    resultado_b = {}
    
    for row in dataset:
        criado = row[t_index]
        formatado = dt.datetime.strptime(criado, '%m/%d/%Y %H:%M')
        hour = formatado.hour
        n_comentarios = int(row[m_index])
        
        if hour in resultado_a:
            resultado_a[hour] +=1
            resultado_b[hour] +=n_comentarios
        else:
            resultado_a[hour] = 1
            resultado_b[hour] = n_comentarios
    
    return ordenar_hora(resultado_a), ordenar_hora(resultado_b)#classifica cada dicionário de resultados por hora

Agora vamos calcular as estatísticas das postagens Ask HN e Show HN:

In [11]:
# armazena o número de posts e comentários do ask por hora em variáveis separadas
ask_h_h_p, ask_h_h_c = calcular_estat_tempo(ask_posts, 6,4)

# armazena o número de posts e comentários do show por hora em variáveis separadas
show_h_h_p, show_h_h_c = calcular_estat_tempo(show_posts, 6,4)

In [24]:
print(ask_h_h_p)
print(show_h_h_p)

{0: 55, 1: 60, 2: 58, 3: 54, 4: 47, 5: 46, 6: 44, 7: 34, 8: 48, 9: 45, 10: 59, 11: 58, 12: 73, 13: 85, 14: 107, 15: 116, 16: 108, 17: 100, 18: 109, 19: 110, 20: 80, 21: 109, 22: 71, 23: 68}
{0: 31, 1: 28, 2: 30, 3: 27, 4: 26, 5: 19, 6: 16, 7: 26, 8: 34, 9: 30, 10: 36, 11: 44, 12: 61, 13: 99, 14: 86, 15: 78, 16: 93, 17: 93, 18: 61, 19: 55, 20: 60, 21: 47, 22: 46, 23: 36}


Os posts Ask são mais comuns às 15:00, 19:00, 21:00, 18:00 e 16:00, e os posts Show são mais comuns às 13:00, 16:00, 17:00 14:00 e 15:00. Geralmente, a cada hora, há mais usuários fazendo perguntas do que aqueles que estão mostrando um conteúdo específico para a comunidade. 
Apenas às 13:00, vemos o maior aumento no número de postagens de HN de shows.

Em seguida, vamos visualizar o número total de comentários por hora das postagens Ask HN e Show HN.

In [25]:
print(ask_h_h_c)
print(show_h_h_c)

{0: 447, 1: 683, 2: 1381, 3: 421, 4: 337, 5: 464, 6: 397, 7: 267, 8: 492, 9: 251, 10: 793, 11: 641, 12: 687, 13: 1253, 14: 1416, 15: 4477, 16: 1814, 17: 1146, 18: 1439, 19: 1188, 20: 1722, 21: 1745, 22: 479, 23: 543}
{0: 487, 1: 246, 2: 127, 3: 287, 4: 247, 5: 58, 6: 142, 7: 299, 8: 165, 9: 291, 10: 297, 11: 491, 12: 720, 13: 946, 14: 1156, 15: 632, 16: 1084, 17: 911, 18: 962, 19: 539, 20: 612, 21: 272, 22: 570, 23: 447}


Podemos ver que, geralmente, as postagens do Ask recebem mais comentários do que as postagens do Show. Com base na visualização acima, é fácil supor que as pessoas que fazem perguntas são mais propensas a receber comentários em suas postagens do que as pessoas que exibem um conteúdo específico.
As horas com o maior número de comentários para as postagens do Ask são 15:00, 16:00, 21:00, 20:00 e 18:00, enquanto as horas que parecem promissoras para garantir os comentários da postagem são 14:00, 16:00, 18:00, 13:00 e 17:00.

Vamos explorar mais e tentar calcular o número médio de comentários por hora para as postagens Ask e Show.

## Média de comentários por hora

Tenhamos em mente a relação:

Média de comentários por hora = Número total de comentários por hora / Número de postagens por hora

Vamos definir uma função chamada `calc_med()` para calcular o número médio de comentários para as postagens Ask HN e Show HN:

In [26]:
def calc_med(dic_a, dic_b):
    resultado = []
    
    for key in dic_a:
        media = dic_b[key]/dic_a[key]
        media = round(media) # arredonda os resultados para o número inteiro mais próximo
        resultado.append([key, media])
        
    return resultado

In [27]:
med_ask_c = calc_med(ask_h_h_p, ask_h_h_c)
med_show_c = calc_med(show_h_h_p, show_h_h_c)

In [28]:
print(med_ask_c)
print(med_show_c)

[[0, 8], [1, 11], [2, 24], [3, 8], [4, 7], [5, 10], [6, 9], [7, 8], [8, 10], [9, 6], [10, 13], [11, 11], [12, 9], [13, 15], [14, 13], [15, 39], [16, 17], [17, 11], [18, 13], [19, 11], [20, 22], [21, 16], [22, 7], [23, 8]]
[[0, 16], [1, 9], [2, 4], [3, 11], [4, 10], [5, 3], [6, 9], [7, 12], [8, 5], [9, 10], [10, 8], [11, 11], [12, 12], [13, 10], [14, 13], [15, 8], [16, 12], [17, 10], [18, 16], [19, 10], [20, 10], [21, 6], [22, 12], [23, 12]]


Observando cuidadosamente, notamos que quando as postagens Ask registram mais comentários (15:00, 20:00, 16:00 e 02:00), elas superam significativamente aquelas vistas com postagens Show. No entanto, quando as postagens Show registram mais comentários, eles são apenas um pouco maiores do que os registrados ao mesmo tempo para as postagens do Ask

Então, como podemos obter informações sobre as cinco principais horas para criar postagens em ordem decrescente, sem usar gráficos ou visualizações complexas? Vamos definir duas funções que nos ajudam a atingir esse objetivo abaixo:

## Classificando e Imprimindo de Listas de Listas

A `ord_lista()` receberá nossas listas contendo a média de comentários Show e Ask e retornará uma lista de médias de comentários em ordem decrescente:

In [31]:
def ord_lista(lista_de_lista):
    
    lista_temp = []
    resultado = []

    for row in lista_de_lista:
        lista_temp.append([row[1], row[0]])

    lista_dec = sorted(lista_temp, reverse=True)
    
    for row in lista_dec:
        resultado.append([row[1], row[0]])
    
    return resultado

In [32]:
# classifica as listas 

com_ask_ord = ord_lista(med_ask_c)
com_show_ord = ord_lista(med_show_c)

Finalmente, a função `print_melhores_horas()` imprimirá as melhores horas para criar postagens Ask ou Show de nossa lista ordenada de listas:

In [35]:
def print_melhores_horas(dataset, medida, post_nome):
    
    print('Top 5 horas para {}  {}:'. format(medida,post_nome))
    print('-'*45)

    contador = 0

    for row in dataset:
        if contador < 5:
            template = '{}: media {} por post - {}'
            valor = row[1]
            horas = row[0]
            horas = dt.time(hour=horas)
            template = template.format(horas,medida,valor)
            print(template)
            contador +=1
        else:
            break

In [36]:
print_melhores_horas(com_ask_ord, 'comentários', 'Ask HN')

print('\n')
print_melhores_horas(com_show_ord, 'comentários', 'Show HN')

Top 5 horas para comentários  Ask HN:
---------------------------------------------
15:00:00: media comentários por post - 39
02:00:00: media comentários por post - 24
20:00:00: media comentários por post - 22
16:00:00: media comentários por post - 17
21:00:00: media comentários por post - 16


Top 5 horas para comentários  Show HN:
---------------------------------------------
18:00:00: media comentários por post - 16
00:00:00: media comentários por post - 16
14:00:00: media comentários por post - 13
23:00:00: media comentários por post - 12
22:00:00: media comentários por post - 12


Ora, o número médio de comentários Ask excede os das postagens Show nas 5 principais horas exibidas acima. 
Às 15:00, as postagens Ask recebem mais de 100% de comentários, em média, do que as médias mais altas obtidas com postagens Show a qualquer hora do dia! Esta informação reforça algumas das observações que fizemos anteriormente.

Assim, o melhor horário para criar postagens Ask é às 15:00. Os usuários que desejam compartilhar conteúdo com a comunidade na forma de postagens Show podem considerar criar essas postagens às 18:00 ou 00:00.

## Conclusão

Neste projeto, analisamos os dados do Hacker News para determinar quais postagens geram mais engajamento. Nós nos concentramos principalmente nas categorias Ask HN e Show HN. 
Com base em nossa análise, recomendamos que os usuários criem postagens do Ask por volta das 15:00. 

O comportamento observado com as postagens Ask e postagens Show pode ser devido, em parte, ao fato de as postagens Ask serem majoritariamente perguntas. As perguntas acenam para que outras pessoas da comunidade forneçam suporte, que geralmente seria fornecido na forma de comentários ou respostas. Mostrar postagens, por outro lado, exigiria alguma criatividade para capturar a multidão, e os usuários que postarem algo interessante provavelmente serão recompensados com votos positivos que determinam o número total de pontos a longo prazo.